# 📌 Implementazione di una Pipeline RAG con FAISS e LangChain

Questo notebook illustra la costruzione di una pipeline **Retrieval-Augmented Generation (RAG)** utilizzando:

- **FAISS** per l'archiviazione e il recupero di embedding vettoriali.
- **LangChain** per il caricamento di documenti, il processamento del testo e l'interrogazione di un LLM.
- **Ollama** come modello LLaMA 3.2 per la generazione delle risposte.
- **Hugging Face** per la generazione degli embedding con `sentence-transformers/all-mpnet-base-v2`.

Ogni sezione include il codice e una spiegazione dettagliata.

---

## 📌 1. Importazione delle librerie

```python
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain_ollama import OllamaLLM
from langchain.chains import RetrievalQA
```

### 📌 Analisi riga per riga
- **`PyPDFLoader`**: Carica e legge documenti PDF.
- **`CharacterTextSplitter`**: Suddivide il testo del PDF in segmenti più piccoli (**chunk**).
- **`HuggingFaceEmbeddings`**: Genera embedding numerici per i chunk di testo.
- **`FAISS`**: Database vettoriale che salva gli embedding e permette la ricerca semantica.
- **`OllamaLLM`**: Interfaccia per interrogare il modello **LLaMA 3.2**.
- **`RetrievalQA`**: Unisce retrieval e generazione in una pipeline automatizzata.

### 📌 Ruolo nel sistema RAG
- **FAISS** archivia gli embedding e supporta la ricerca vettoriale.
- **Hugging Face** genera gli embedding per confrontare documenti simili.
- **LangChain** gestisce il caricamento dei documenti e la suddivisione in chunk.
- **Ollama** elabora le domande dell'utente e genera risposte in linguaggio naturale.

---

## 📌 2. Caricamento del documento

```python
# Load the document
loader = PyPDFLoader("Foundations of LLMs.pdf")
documents = loader.load()
```

### 📌 Cosa fa questa riga?
- **`PyPDFLoader("Foundations of LLMs.pdf")`** carica il file PDF.
- **`documents = loader.load()`** legge il testo e lo trasforma in una lista di oggetti LangChain.

### 📌 Ruolo nel sistema RAG
- Estrae contenuto da un documento per poterlo indicizzare e successivamente interrogare.

---

## 📌 3. Suddivisione del documento in chunk

```python
# Split the document into chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=30, separator="
")
docs = text_splitter.split_documents(documents=documents)
```

### 📌 Cosa fa questa riga?
- **`chunk_size=1000`**: Divide il testo in blocchi da **1000 caratteri**.
- **`chunk_overlap=30`**: Sovrappone i chunk di **30 caratteri** per mantenere il contesto.
- **`separator="
"`**: Divide il testo basandosi sui ritorni a capo (`
`).

### 📌 Ruolo nel sistema RAG
- Suddividere il documento è essenziale per la ricerca semantica: aiuta FAISS a gestire e recuperare informazioni rilevanti.

---

## 📌 4. Creazione degli embedding con Hugging Face

```python
# ✅ Impostiamo il modello per funzionare sulla CPU
embedding_model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {"device": "cpu"}  # 🔴 Cambia "cuda" in "cpu"

# ✅ Caricamento del modello
embeddings = HuggingFaceEmbeddings(
    model_name=embedding_model_name,
    model_kwargs=model_kwargs
)
```

### 📌 Cosa fa questa riga?
- **`sentence-transformers/all-mpnet-base-v2`** è un modello ottimizzato per il **retrieval semantico**.
- **`device: "cpu"`** imposta il modello per funzionare sulla CPU.

### 📌 Ruolo nel sistema RAG
- Gli embedding permettono di **trasformare il testo in vettori numerici**, essenziali per la ricerca nei database vettoriali.

---

## 📌 5. Creazione dello store vettoriale FAISS

```python
# ✅ Creazione dello store vettoriale FAISS
vectorstore = FAISS.from_documents(docs, embeddings)
```

### 📌 Cosa fa questa riga?
- **`FAISS.from_documents(docs, embeddings)`**:
  - Converte ogni chunk in un embedding.
  - Li salva in una struttura ottimizzata per la ricerca vettoriale.

### 📌 Ruolo nel sistema RAG
- FAISS permette di **ricercare il documento più simile a una query** basandosi sulla distanza vettoriale.

---

## 📌 6. Salvataggio e caricamento dello store FAISS

```python
# ✅ Salvataggio e caricamento dello store vettoriale
vectorstore.save_local("faiss_index_")
persisted_vectorstore = FAISS.load_local("faiss_index_", embeddings, allow_dangerous_deserialization=True)
```

### 📌 Cosa fa questa riga?
- **`save_local("faiss_index_")`** salva FAISS su disco per un uso futuro.
- **`load_local("faiss_index_")`** ricarica FAISS in memoria.

### 📌 Ruolo nel sistema RAG
- Permette di **evitare il ricalcolo degli embedding**, ottimizzando le prestazioni.

---

## 📌 7. Creazione del retriever

```python
retriever = persisted_vectorstore.as_retriever()
```

### 📌 Cosa fa questa riga?
- Converte FAISS in un **retriever**, permettendo di recuperare documenti in base a una query.

### 📌 Ruolo nel sistema RAG
- Gestisce l’interrogazione del database vettoriale per trovare i documenti più pertinenti.

---

## 📌 8. Inizializzazione del modello LLaMA

```python
# Initialize the LLaMA model
llm = OllamaLLM(model="llama3.2")
```

### 📌 Cosa fa questa riga?
- Carica il modello **LLaMA 3.2** tramite Ollama.

### 📌 Ruolo nel sistema RAG
- **Genera le risposte basandosi sulle informazioni recuperate dal retriever**.

---

## 📌 9. Creazione della pipeline RAG con `RetrievalQA`

```python
# ✅ Crea il modello di QA con il retriever
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)
```

### 📌 Cosa fa questa riga?
- **Unisce retrieval e generazione** in una pipeline automatizzata.

### 📌 Ruolo nel sistema RAG
- Recupera documenti rilevanti **e costruisce il prompt** per il modello LLaMA automaticamente.

---

## 📌 10. Loop interattivo per interrogare il sistema

```python
while True:
    query = input("Type your query (or type 'Exit' to quit): 
")
    if query.lower() == "exit":
        break
    
    # 🔴 Sostituire .run() con .invoke()
    result = qa.invoke(query)  # ✅ Metodo corretto
    print(result)
```

### 📌 Cosa fa questa riga?
- **Chiede all'utente una domanda**.
- **Recupera i documenti pertinenti**.
- **Passa il prompt a LLaMA** e **genera una risposta**.

### 📌 Ruolo nel sistema RAG
- Permette di **interrogare il modello** con domande in linguaggio naturale.

---

🚀 **Ora la pipeline è pronta per essere testata!**

In [1]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter

# Load the document
loader = PyPDFLoader("Foundations of LLMs.pdf")
documents = loader.load()

# Split the document into chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=30, separator="\n")
docs = text_splitter.split_documents(documents=documents)

In [3]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

# ✅ Impostiamo il modello per funzionare sulla CPU
embedding_model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {"device": "cpu"}  # 🔴 Cambia "cuda" in "cpu"

# ✅ Caricamento del modello
embeddings = HuggingFaceEmbeddings(
    model_name=embedding_model_name,
    model_kwargs=model_kwargs
)

# ✅ Creazione dello store vettoriale FAISS
vectorstore = FAISS.from_documents(docs, embeddings)

# ✅ Salvataggio e caricamento dello store vettoriale
vectorstore.save_local("faiss_index_")
persisted_vectorstore = FAISS.load_local("faiss_index_", embeddings, allow_dangerous_deserialization=True)

# ✅ Creazione del retriever
retriever = persisted_vectorstore.as_retriever()

In [6]:
from langchain_ollama import OllamaLLM

# Initialize the LLaMA model
llm = OllamaLLM(model="llama3.2")

# Test with a sample prompt
response = llm.invoke("Tell me a joke")
print(response)

Here's one:

What do you call a fake noodle?

An impasta.


In [10]:
from langchain.chains import RetrievalQA

# ✅ Crea il modello di QA con il retriever
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)

# ✅ Loop interattivo per interrogare il sistema
while True:
    query = input("Type your query (or type 'Exit' to quit): \n")
    if query.lower() == "exit":
        break
    
    # 🔴 Sostituire .run() con .invoke()
    result = qa.invoke(query)  # ✅ Metodo corretto
    print(result)

{'query': 'What is long sequence modeling?', 'result': 'Long sequence modeling refers to the process of analyzing or generating sequences that are significantly longer than what can be accommodated by traditional models. In the context of language models, this often involves dealing with very large input sequences, such as text documents or conversations, where each token represents a word or character in the sequence.'}
{'query': 'what is model ensembling?', 'result': 'According to the context provided, Model Ensembling (option a) refers to a method where multiple LLMs varying in architectures or parameters are used. Each LLM receives the same prompt and produces a prediction, which are then combined to generate the final prediction.'}
